<a href="https://colab.research.google.com/github/StefanPushkov/Stepik_MIPT/blob/master/Copy_of_%5Bhomework%5Dseq2seq_and_attn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hausaufgabe
translate(Hausaufgabe) -> **Homework**

Halo!
На семинаре мы создали простую seq2seq модель на основе rnn для перевода, а сейчас постараемся засунуть туда attention. Работать будем с тем же датасетом DE->EN (датасеты получше просто не влезают в память колаба, но если у вас есть CPU+тонна времени или GPU побольше, то можно попробовать построить перевод на WMT14 или IWSLT )

В конце домашней работы предполагается написание отчета о проделанной работе.

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import spacy

import random
import math
import time

from torchtext.datasets import TranslationDataset, Multi30k #WMT14, IWSLT
from torchtext.data import Field, BucketIterator

import torch.nn.functional as F

In [0]:
seed = 43

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
! python -m spacy download en
! python -m spacy download de


spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 824kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907056 sha256=cd0280e6c6069f882089ddaa5d080764a635030a718fe91c6ea1182aad82d838
  Stored in directory: /tmp/pip-ephem-wheel-cache-w841ts4h/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [0]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

# немецкий язык является полем SRC, а английский в поле TRG
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [6]:
# В датасете содержится ~ 30к предложений средняя длина которых 11
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),  fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 569kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 171kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 166kB/s]


Давайте посмотрим что у нас с датасетом и сделаем словари для SRC и TGT

In [7]:
labels = ['train', 'validation', 'test']
dataloaders = [train_data, valid_data, test_data]
for d, l in zip(dataloaders, labels):
    print("Number of sentences in {} : {}".format(l, len(d.examples)))


Number of sentences in train : 29000
Number of sentences in validation : 1014
Number of sentences in test : 1000


In [8]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print("Number of words in source vocabulary", len(SRC.vocab))
print("Number of words in source vocabulary", len(TRG.vocab))

Number of words in source vocabulary 7855
Number of words in source vocabulary 5893


## Encoder

Энкодер будет ровно как в семинаре, с кдинственным изменением -- forward будет возвращать не только hidden, cell, но еще и outputs. Это нужно (надеюсь, вы уже поняли) для использования attention в декодере

In [0]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        """
        :param: input_dim is the size/dimensionality of the one-hot vectors that will be input to the encoder. This is equal to the input (source) vocabulary size.
        :param: emb_dim is the dimensionality of the embedding layer. This layer converts the one-hot vectors into dense vectors with emb_dim dimensions.
        :param: hid_dim is the dimensionality of the hidden and cell states.
        :param: n_layers is the number of layers in the RNN.
        :param: percentage of the dropout to use
        
        """
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
 
        self.rnn = nn.GRU(emb_dim, hid_dim, self.n_layers,
                          dropout=dropout,
                          bidirectional=True)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        """
        :param: src sentences (src_len x batch_size)
        """
        # embedded = <TODO> (src_len x batch_size x embd_dim)
        embedded = self.embedding(src)
        # dropout over embedding
        embedded = self.dropout(embedded)
        outputs, hidden= self.rnn(embedded)

        # sum bidirectional outputs
        outputs = (outputs[:, :, :self.hid_dim] +
                   outputs[:, :, self.hid_dim:])
        # [Attention return is for lstm, but you can also use gru]
        return outputs, hidden

## Decoder
Оп ля, а тут уже что-то новенькое

Мы будем реализовывать attention, который будет смотреть из tgt в src (НЕ self-attention). 

Определим два класса -- Attention и DecoderAttn. Мы разбили их на два класса, чтобы можно было играться с типом внимания, не меняя код DecoderAttn. Как вы помните с лекции, в качестве аттеншена можно брать любую странную функцию (конкатенация, маленькая сеточка, ...), и все будет работать! Поэтому вам предлагается попробовать несколько разных.


---------------------
Есть два подхода к реализации аттеншена:

Подход #1:

1. Вычисляется embed
2. На основе hidden c прошлого шага, embedded и (возможно) enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!). Размерность batch_size * max_len, max_len -- максимальная длина предложения в батче, т.е. shape[0] от выхода энкодера.
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Берутся attention и embedded и сворачиваются в один вектор размерности такой, чтобы кормить его self.lstm. Например, это можно сделать с помощью обычного линейного слоя
5. Вычисляется новое скрытое состояние new_hidden. Это наша self.lstm, примененная к выходу пункта 4.
6. Вычисляется prediction, как в семинаре

Грубо говоря, вся разница с семинаром в том, что мы вместо того, чтобы embedded пихать в self.lstm, миксуем аттэншен на основе всего, что имеем (enc_out, hidden, embedded) и запихиваем в self.lstm микс аттэншена и embedded.

![alt text](https://i.imgur.com/cmkRY0r.png)


Подход #2:

1. Вычисляется embed
2. Вычисляется output, new_hidden (строчка output, (hidden, cell) = self.rnn(embedded, (hidden, cell)))
3. На основе output и enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!)
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Вычисляется prediction на основе attention и output. Можно, например, взять nn.Linear() от конкатенации attention и output.

Разница с первым подходом в том, что мы сначала вычисляем выход rnn слоя, а потом смотрим вниманием на src и на основе выхода rnn и attn считаем выход (prediction). 

![alt text](https://i.imgur.com/5aWjQWv.png)


Вам предлагается реализовать хотя бы 1 из вариантов и хотя бы 2 варианта функции attention (в классе Attention)


In [0]:
# b-etienne

class Attention(nn.Module):
    def __init__(self, hidden_size, method="one-layer-net"): # add parameters needed for your type of attention
        super(Attention, self).__init__()
        self.method = method # attention method you'll use. e.g. "cat", "one-layer-net", "dot", ...
        self.hidden_size = hidden_size
        self.v = nn.Parameter(torch.rand(hidden_size))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)

        if self.method == 'dot':
            pass
        elif self.method == 'one-layer-net':
            self.Xnet = nn.Linear(hidden_size* 2, hidden_size)
        else:
            raise NotImplementedError
        
    def forward(self, last_hidden, encoder_outputs, seq_len=None):

        
        
        if self.method == 'dot':
            # last_hidden = last_hidden.unsqueeze(-1)
            return encoder_outputs.bmm(last_hidden).squeeze(-1)

        elif self.method == 'one-layer-net':
            timestep = encoder_outputs.size(0)
            h = last_hidden.repeat(timestep, 1, 1).transpose(0, 1)
            encoder_outputs = encoder_outputs.transpose(0, 1)  # [B*T*H]
            attn_energies = self.score(h, encoder_outputs)
            return F.softmax(attn_energies, dim=1).unsqueeze(1)
        else:
            raise NotImplementedError

    def score(self, hidden, encoder_outputs):
        # [B*T*2H]->[B*T*H]
        energy = F.relu(self.Xnet(torch.cat([hidden, encoder_outputs], 2)))
        energy = energy.transpose(1, 2)  # [B*H*T]
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # [B*1*H]
        energy = torch.bmm(v, energy)  # [B*1*T]
        return energy.squeeze(1)  # [B*T]


class DecoderAttn(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout=0.1):
        super(DecoderAttn, self).__init__()
        
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        
        self.attn = Attention(self.hid_dim, method="one-layer-net") # instance of Attention class

        # define layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.gru = nn.GRU(self.hid_dim + self.emb_dim, self.hid_dim ,
                          n_layers, dropout=dropout)
         
        self.out = nn.Linear(self.hid_dim * 2, self.output_dim) # Projection :hid_dim x output_dim
        self.dropout = nn.Dropout(dropout)

        
      
    def forward(self, input, last_hidden, encoder_output):
        
        # make decoder with attention
        # use code from seminar notebook as base and add attention to it

        # (1x batch_size)
        input = input.unsqueeze(0)

        # (1 x batch_size x emb_dim)
        embedded = self.embedding(input)# embd over input and dropout 
        embedded = self.dropout(embedded)

        # Calculate attention weights and apply to encoder outputs
        attn_weights = self.attn(last_hidden[-1], encoder_output)
        context = attn_weights.bmm(encoder_output.transpose(0, 1))  # (B,1,N)
        context = context.transpose(0, 1)  # (1,B,N)

        # Combine embedded input word and attended context, run through RNN
        rnn_input = torch.cat([embedded, context], 2)
        output, hidden = self.gru(rnn_input, last_hidden)
        output = output.squeeze(0)  # (1,B,N) -> (B,N)
        context = context.squeeze(0)
        output = self.out(torch.cat([output, context], 1))
        output = F.log_softmax(output, dim=1)
        return output, hidden, attn_weights
      
    # def forward(self, input, hidden, cell, encoder_output):
    #     # make decoder with attention
    #     # use code from seminar notebook as base and add attention to it
        
    #     # 1. Get emb
    #     embedded = self.embedding(input).unsqueeze(1)

    #     # 1.1 Apply dropout
    #     # embedded = self.dropout(embedded)
        
    #     hidden = hidden.unsqueeze(0)
    #     # 2. Calculate output, new_hidden: строчка output, (hidden, cell) = self.rnn(embedded, (hidden, cell)) # rnn_input = torch.cat((embedded, hidden.unsqueeze(0)), 2)
    #     rnn_output, hidden = self.rnn(embedded, hidden) #self.rnn(embedded, (hidden, cell))
    #     rnn_output = rnn_output.squeeze(1)

    #     # 3. Calculating attention weights
    #     weights = self.attn.forward(rnn_output, encoder_output) # B x T
    #     context = weights.unsqueeze(1).bmm(encoder_output).squeeze(1)

    #     # 4. 
    #     output = self.character_distribution(torch.cat((rnn_output, context), 1))
    #     prediction = F.log_softmax(output, -1)


        
    #     return prediction, hidden.squeeze(0), weights # hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
from torch.autograd import Variable

## Seq2Seq module

Здесь опять ничего не поменяется кроме того, что энкодер теперь возвращает свой output, а декодер его принимает

In [0]:
# we need that to put it first to the decoder in 'translate' method
BOS_IDX = SRC.vocab.stoi['<sos>']

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        # Hidden dimensions of encoder and decoder must be equal
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self._init_weights() 
        self.max_len=30
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        """
        :param: src (src_len x batch_size)
        :param: tgt
        :param: teacher_forcing_ration : if 0.5 then every second token is the ground truth input
        """
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.tensor(torch.zeros(max_len, batch_size, trg_vocab_size)).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        enc_out, hidden = self.encoder(src)
        hidden = hidden[:self.decoder.n_layers] # !!!!!!!!!!!
        
        #first input to the decoder is the <sos> tokens
        input_ = Variable(trg.data[0, :])
        
        for t in range(1, max_len):
            
            output, hidden, attn_weights = self.decoder(
                    input_, hidden, enc_out) #TODO pass state and input throw decoder 
            # print('outputs shape: {0}\noutput shape: {1}'.format(outputs.shape, output.shape))
            outputs[t] = output
            
            is_teacher = random.random() < teacher_forcing_ratio
            top1 = output.data.max(1)[1]
            input_ = torch.tensor(trg.data[t] if is_teacher else top1)
        
        return outputs
        
    def translate(self, src):
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = []
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        src = src.unsqueeze(0).to(self.device)
        # print("Src shape before encoder: {0}".format(src.shape))
        enc_out, hidden = self.encoder(src) # TODO pass src throw encoder
        # print('Hidden before slice: {0}'.format(hidden.shape))
        hidden = hidden[:self.decoder.n_layers]
        # print('Hidden after slice: {0}'.format(hidden.shape))

        #first input to the decoder is the <sos> tokens
        batch_size = src.shape[1]
        input_ =  torch.tensor([BOS_IDX] * batch_size).to(self.device) # [SRC.vocab.stoi['<sos>'] for _ in range(len(src))]
        # print('input_ in translate method: {0}'.format(input_.shape))
        # input_ = torch.tensor([TRG.vocab.stoi['<sos>']])
        # print('input_ shape {0}'.format(input_.shape))
        # .squeeze(-1)# TODO trg[idxs]
        
        for t in range(1, self.max_len):
            
            output, hidden, attn_weights = self.decoder(
                    input_, hidden, enc_out) #TODO pass state and input throw decoder 
            top1 = output.max(1)[1]
            outputs.append(top1)
            input_ = (top1)
        
        return outputs
    
    def _init_weights(self):
        p = 0.08
        for name, param in self.named_parameters():
            nn.init.uniform_(param.data, -p, p)
        


In [0]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
src_embd_dim =  tgt_embd_dim = 256
hidden_dim = 512
num_layers =  2
dropout_prob = 0.2


batch_size = 64
PAD_IDX = TRG.vocab.stoi['<pad>']

iterators = BucketIterator.splits((train_data, valid_data, test_data),
                                  batch_size = batch_size, device = device)
train_iterator, valid_iterator, test_iterator = iterators

enc = Encoder(input_dim, src_embd_dim, hidden_dim, num_layers, dropout_prob)
dec = DecoderAttn(output_dim, tgt_embd_dim,hidden_dim, num_layers, dropout=dropout_prob)
model = Seq2Seq(enc, dec, device).to(device)


In [53]:
model


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): GRU(256, 512, num_layers=2, dropout=0.2, bidirectional=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): DecoderAttn(
    (attn): Attention(
      (Xnet): Linear(in_features=1024, out_features=512, bias=True)
    )
    (embedding): Embedding(5893, 256)
    (gru): GRU(768, 512, num_layers=2, dropout=0.2)
    (out): Linear(in_features=1024, out_features=5893, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

In [0]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        # print(batch.src.shape)
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing !!
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)


            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [56]:
max_epochs = 13
CLIP = 1

# TODO
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

best_valid_loss = float('inf')

for epoch in range(max_epochs):
    
    
    train_loss = round(train(model, train_iterator, optimizer, criterion, CLIP), 5)
    valid_loss = round(evaluate(model, valid_iterator, criterion),5)
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    print('Train Perplexity {}  Val Perplexity {}:'.format(np.exp(train_loss), np.exp(valid_loss)))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


KeyboardInterrupt: ignored

In [0]:
# model = Model
# model.load_state_dict(torch.load('model.pt'))
# test_loss = evaluate(model, test_iterator, criterion)


# print('| Test Loss: {} Test PPL:{}|'.format(test_loss, np.exp(test_loss)))

In [237]:
test_loss = evaluate(model, test_iterator, criterion)

print('| Test Loss: {} Test PPL:{}|'.format(test_loss, np.exp(test_loss)))

| Test Loss: 3.1742746382951736 Test PPL:23.90947056254606|


In [0]:
EOS_IDX = SRC.vocab.stoi['<eos>']

def translate(sentence):
    """
    function that uses .translate() method of the model to translate german sentence into english
    params:
        sentence: tokenized gernam sentence
    """
    sentence = sentence.lower()
    sent_vec = [SRC.vocab.stoi[token] for token in sentence.split()]
    answ = ''
    translation_idx = model.translate(torch.tensor(sent_vec))
    for t in translation_idx:
        if t[0] != EOS_IDX:
            print(TRG.vocab.itos[t[0]], end=' ')
            answ += TRG.vocab.itos[t[0]]
            answ += ' ' 
        else:
            break

    return answ.replace(".", "")

In [241]:
translate("ein klein apfel")

a small hat is taking a pond . 

'asmallhatistakingapond'

ИИИИ давайте также научимся считать самую популярную метрику для перевода -- BLEU (https://en.wikipedia.org/wiki/BLEU)

В общем-то, вам повезло -- ее писать руками скучно, да и nltk ее написало за вас:

In [0]:
from nltk.translate.bleu_score import corpus_bleu
def compute_bleu(inp_lines, out_lines):
    """ Estimates corpora-level BLEU score of model's translations given inp and reference out """
    print(inp_lines.split())
    translations = translate(inp_lines)
    print('translations: ', translations)
    print([out_lines.split()], translations.split())
    return corpus_bleu([[out_lines.split()]], [translations.split()]) * 100

In [230]:

compute_bleu('ein klein apfel', 'a small apple')

['ein', 'klein', 'apfel']
a snowboarder is taking a large gun while a large gun . translations:  a snowboarder is taking a large gun while a large gun  
[['a', 'small', 'apple']] ['a', 'snowboarder', 'is', 'taking', 'a', 'large', 'gun', 'while', 'a', 'large', 'gun']


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


54.91004867761124

In [212]:
import nltk
original = 'a small apple'
hypothesis = 'a small apple'
original_split = original.split()
hypothesis_split = hypothesis.split()

print([original_split], hypothesis_split)
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([original_split], [hypothesis_split])
print(BLEUscore)

[['a', 'small', 'apple']] ['a', 'small', 'apple']
1.0


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Если вы реализовали несколько методов аттеншена, опишите каждый из них и полученную метрику на нем в отчете.

<Место для вашего отчета>